In [ ]:
# Importing the libraries

import os
import sys



import pycharmm
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.lingo as lingo

from pycharmm.lib import charmm as libcharmm

In [ ]:
# Load the topology files using pyCHARMM  
read.stream('toppar.str')

In [ ]:
# Load the psf file using psf_card in pyCHARMM

read.psf_card('traj.psf')
psf.get_nres()

In [ ]:
#Define the input and output files
#Input: trajectory file
#Output: order parameter file

indcd = 'traj.dcd' 
outdat = 'order.dat'


#CHARMM units are used to read and write files
pycharmm.lingo.charmm_script('open unit 21 write form name ' + outdat ) 
pycharmm.lingo.charmm_script('open unit 62 read unform name ' + indcd )

#Print Trajectory Information
pycharmm.lingo.charmm_script('TRAJectory QUERy UNIT 62') 

In [ ]:
#CHARMM STtream File Generation for NMR Module, Set options for NMR module
nmr_stream = '''* NMR Order Parameter Calculation       !!Stream File Title
*                                                       !! Syntax Requirement
                                                        !! Blank line, Syntax Requirement    
NMR                                                     !! NMR module Start    
RESEt                                                   !! Reset NMR module
RTIMes sele atom * * N end sele atom * * HN end         !! Select the atoms to calculate the order parameter
DYNA firstu 62 -                                        !! unit number to read from (same as dcd unit)
     nunit 1 -                                          !! number of units to read
     begin 1 -                                          !! first frame to read
     stop 1124 -                                        !! last frame to read
     skip 1 -                                           !! read every skip frames
     tmax 3.0 -                                         !! maximum time for correlation function
     cut 2.0 -                                          !! cut-off for correlation function
     ilist 21 -                                         !! unit number to write to (same as outdat unit)
     dsigma -160.0 C(t) !                               !! correlation function
END                                                     !! End of NMR module ! More option can be seen from the documentation
'''
with open('nmr.str', 'w') as f:
    f.write(nmr_stream)

# Running NMR Module via read option in pyCHARMM
read.stream('nmr.str')

In [ ]:
#Plotting the RESI vs <S2> plot from the order.dat file

import numpy as np
import matplotlib.pyplot as plt

# Load data from 'order.dat', skipping the header row
data = np.genfromtxt('order.dat', skip_header=1, usecols=(0, 6), dtype=float, filling_values=np.nan)

# Remove rows with NaN values (rows that had non-numeric data)
data = data[~np.isnan(data).any(axis=1)]

# Extract 'resi' and '<S2>' columns from the data
resi_values = data[:, 0]  # Assuming 'resi' is in the first column (index 0)
s2_values = data[:, 1]     # Assuming '<s2>' is in the second column (index 1)

# Plotting using matplotlib
plt.plot(resi_values, s2_values, marker='o')
plt.xlabel('RESI')
plt.ylabel('<S2>')
plt.title('RESI vs <S2> Plot')
plt.grid(True)
plt.show()